##### Plan and Execute

this notebook shows how to create 'plan-and-execute' style agent. This is heavily inspired by the plan-and-solve paper as well as babyAGI. 
The core idea is to first come up with a multi-step plan, and then go through that plan one item at a time. After accomplishing a particular task, you can then revisit the plan and modify as appropriate. This compares to a typical ReAct style agent where you are thinking one step at a time. Teh advantanges of plan-and-execute style agents are:
- Explicit long term planning
- Ability to use smaller/weaker models for the execturion step, only using larger/better models for the planning step

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

##### Define Tools
We will first define the tools we want to use. For this simple example, we will use a builti in search tool via Tavily. However, it is really easy to create your own tools.

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults
tools = [TavilySearchResults(max_results=3)]

#### Defin our execution agent
Now we will create the execution agent we want to use to execute tasks. Note that for this example, we will be using the same execution agent for each task, although this is not a requirement

In [3]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai import ChatOpenAI


prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="mistralai/Mixtral-8x7B-Instruct-v0.1")
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

In [4]:
from langgraph.prebuilt import create_agent_executor

agent_executor = create_agent_executor(agent_runnable, tools)

In [5]:
agent_executor.invoke(
    {
        "input": "who is the winner of the us open", "chat_history": []
    }
)

{'input': 'who is the winner of the us open',
 'chat_history': [],
 'agent_outcome': AgentFinish(return_values={'output': ''}, log=''),
 'intermediate_steps': []}

##### Define the state
Let's now start by defining the state the track of this agent.

First, we will need to track the current plan. Let's represent that as a list of strings. 
Next, we should track previously executed steps. Let's represent that as a list tuples, theres tuples will contain the step and then the result.

Finally, we need to have some state to represent the final respons as well as the original input

In [6]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Tuple, Annotated, TypedDict
import operator


class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps = Annotated[List[Tuple], operator.add]
    response: str

##### Planning step
Lets think about creating the planning step. This will use function calling to create a plan

In [7]:
from langchain_core.pydantic_v1 import BaseModel
class Plan(BaseModel):
    steps: List[str] = Field(
        description="different steps to follow, should be in sorted order",
    )

In [8]:
from langchain.chains.openai_functions import create_structured_output_chain
from langchain_core.prompts import ChatPromptTemplate


planner_prompt = ChatPromptTemplate.from_template(
    """
    For the given objective, come up with a simple step by step plan. \
    This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
    The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

    {objective}
    """
)

planner = create_structured_output_chain(Plan, ChatOpenAI(model="mistralai/Mixtral-8x7B-Instruct-v0.1"), planner_prompt)

In [9]:
planner.invoke({
    "objective": "what is the hometown of the current Australia open winnner?"
})

{'objective': 'what is the hometown of the current Australia open winnner?',
 'function': Plan(steps=['Search for the current Australia Open winner', 'Locate their hometown in their bio or profile', 'Return the hometown as the final answer'])}

##### Re-Plan Step
Now, Let's create a step that re-does the plan blased on the result of the previous step

In [11]:
from langchain.chains.openai_functions import create_openai_fn_runnable

class Response(BaseModel):
    """
    Respone the user
    """
    response: str
    
    
replanner_prompt = ChatPromptTemplate.from_template(
    template="""For the given objective, come up with a simple step by step plan
    This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
    The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

    Your objective was this:
    {input}

    Your original plan was this:
    {plan}

    You have currently done the follow steps:
    {past_steps}

    Update your plan accordingly. If no more steps are needed and you can return to the user, then respond with that.
    """
)

replanner = create_openai_fn_runnable(
    [Plan, Response],
    ChatOpenAI(model="mistralai/Mixtral-8x7B-Instruct-v0.1"),
    replanner_prompt,
)

##### Create the Graph
We can now create the graph

In [12]:
async def execute_step(state: PlanExecute):
    task = state["plan"][0]
    agent_response = await agent_executor.ainvoke({"input": task, "chat_history":[]})
    return {
        "past_steps": (task, agent_response["agent_outcome"].return_values["output"])
    }
    
async def plan_step(state: PlanExecute):
    plan = await planner.ainvoke({"objective": state["input"]})
    return {"plan": plan.steps}

async def replan_step(state: PlanExecute):
    output = await replanner.ainvoke({"objective": state["input"]})
    if isinstance(output, Response):
        return {"response": output.response}
    else:
        return {"plan": output.steps}
    
def should_end(state: PlanExecute):
    if state["response"]:
        return True
    else: 
        return False

In [13]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(PlanExecute)

workflow.add_node("planner", plan_step)
workflow.add_node("agent", execute_step)
workflow.add_node("replan", replan_step)

workflow.set_entry_point("planner")

workflow.add_edge("planner", "agent")

workflow.add_edge("agent", "replan")

workflow.add_conditional_edges("replan", should_end, {
    True: END, 
    False: "agent"
})

app = workflow.compile()

In [14]:
from langchain_core.messages import HumanMessage

config = {"recursion_limit": 50}
inputs = {"input": "what is the hometown of the 2024 Australia open winner?"}
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)

AttributeError: 'dict' object has no attribute 'steps'